In [27]:
#libraries
import numpy as np
import pandas as pd
import pickle
from functools import reduce

In [14]:
#Load Data
volumes = pd.read_csv('../temporary/volumes.csv')
industry = pd.read_csv('../input/industry_scores.csv')
sentiment = pd.read_csv('../input/sentiment_scores_march23.csv')

#volume metadata
metadata = pickle.load(open('../input/metadata.p', 'rb'))

metadata['Year'] = pd.to_numeric(metadata['Year'], downcast='signed')

def fix_htid(row):
    return row['HTID'].replace(":","+").replace("/", "=")

metadata['HTID'] = metadata.apply(fix_htid, axis=1)
metadata.drop(columns=['oclc'], inplace=True)

In [15]:
#clean data
industry = industry.rename(columns={'Unnamed: 0': 'HTID', '2-vote':'industry_2','3-vote':'industry_3'})
industry['HTID'] = industry['HTID'].map(lambda x: x.rstrip('.txt'))#remove '.txt' at the end of each string for HTIDs

#Clean Sentiment Data

sentiment = sentiment.rename(columns = {'Unnamed: 0': 'HTID', 'Regression': 'percent_regression', 'Pessimism': 'percent_pessimism', 'Optimism':'percent_optimistic', 'Progress': 'percent_progress'})

sentiment['HTID'] = sentiment['HTID'].map(lambda x: x.rstrip('.txt')) #remove '.txt' at the end of each string for HTIDs


In [16]:
#Dimensions
print('volumes:' + str(volumes.shape))
print('industry:' + str(industry.shape))
print('sentiment:' + str(industry.shape))

volumes:(166780, 4)
industry:(173067, 3)
sentiment:(173067, 3)


In [24]:
sentiment_scores = pd.merge(sentiment, industry, on = 'HTID')
sentiment_scores_metadata = pd.merge(sentiment_scores, metadata, on = 'HTID')
topic_scores_metadata = pd.merge(volumes, metadata, on = 'HTID')

In [25]:
print('sentiment_scores:' + str(sentiment_scores.shape))
print('sentiment_scores_metadata:' + str(sentiment_scores_metadata.shape))
print('topic_scores_metadata:' + str(topic_scores_metadata.shape))

sentiment_scores:(173137, 7)
sentiment_scores_metadata:(168931, 8)
topic_scores_metadata:(166782, 5)


In [39]:
dfs = [sentiment_scores, volumes, metadata]

# final_merge = pd.merge(sentiment_scores_metadata, volumes, on = 'HTID')
final_merge = reduce(lambda left, right: pd.merge(left, right, on = 'HTID', how = 'inner'), dfs)
outer = pd.merge(sentiment_scores, volumes, on = 'HTID', how = 'outer', indicator=True)
anti = outer[~(outer._merge == 'both')]
anti = pd.merge(anti, metadata, on = 'HTID')

In [40]:
anti 

,HTID,percent_regression,percent_pessimism,percent_optimistic,percent_progress,industry_2,industry_3,Religion,Science,Political Economy,_merge,Year
0,nyp.33433081689832,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,left_only,1810.0
1,hvd.hn5cyr,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,left_only,1848.0
2,uc1.$b283481,0.000000,0.000000,0.000000,0.000000,0.013649,0.013649,NaN,NaN,NaN,left_only,1885.0
3,hvd.32044106334329,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,left_only,1794.0
4,uc1.$b13904,0.000253,0.000253,0.000253,0.000253,0.046071,0.050448,NaN,NaN,NaN,left_only,1838.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10206,uiuo.ark+=13960=t3dz0fk4x,NaN,NaN,NaN,NaN,NaN,NaN,0.380205,0.123039,0.496756,right_only,1875.0
10207,uc2.ark+=13960=t4zg6w79x,NaN,NaN,NaN,NaN,NaN,NaN,0.342954,0.111881,0.545165,right_only,1894.0
10208,uc2.ark+=13960=t1sf2nv5x,NaN,NaN,NaN,NaN,NaN,NaN,0.428217,0.051069,0.520715,right_only,1848.0
10209,uiuo.ark+=13960=t3kw5p40t,NaN,NaN,NaN,NaN,NaN,NaN,0.323473,0.052603,0.623924,right_only,1835.0


In [41]:
map = {'left_only':'sentiment_data', 'right_only':'topic_weights'}
anti['dataset'] = anti['_merge'].map(map)
anti

,HTID,percent_regression,percent_pessimism,percent_optimistic,percent_progress,industry_2,industry_3,Religion,Science,Political Economy,_merge,Year,dataset
0,nyp.33433081689832,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,left_only,1810.0,sentiment_data
1,hvd.hn5cyr,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,left_only,1848.0,sentiment_data
2,uc1.$b283481,0.000000,0.000000,0.000000,0.000000,0.013649,0.013649,NaN,NaN,NaN,left_only,1885.0,sentiment_data
3,hvd.32044106334329,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,left_only,1794.0,sentiment_data
4,uc1.$b13904,0.000253,0.000253,0.000253,0.000253,0.046071,0.050448,NaN,NaN,NaN,left_only,1838.0,sentiment_data
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10206,uiuo.ark+=13960=t3dz0fk4x,NaN,NaN,NaN,NaN,NaN,NaN,0.380205,0.123039,0.496756,right_only,1875.0,topic_weights
10207,uc2.ark+=13960=t4zg6w79x,NaN,NaN,NaN,NaN,NaN,NaN,0.342954,0.111881,0.545165,right_only,1894.0,topic_weights
10208,uc2.ark+=13960=t1sf2nv5x,NaN,NaN,NaN,NaN,NaN,NaN,0.428217,0.051069,0.520715,right_only,1848.0,topic_weights
10209,uiuo.ark+=13960=t3kw5p40t,NaN,NaN,NaN,NaN,NaN,NaN,0.323473,0.052603,0.623924,right_only,1835.0,topic_weights


In [42]:
anti_volumes = anti[['HTID', 'Year', 'dataset']]

In [46]:
anti_volumes.to_csv('../temporary/anti_joined.csv')
anti_volumes

,HTID,Year,dataset
0,nyp.33433081689832,1810.0,sentiment_data
1,hvd.hn5cyr,1848.0,sentiment_data
2,uc1.$b283481,1885.0,sentiment_data
3,hvd.32044106334329,1794.0,sentiment_data
4,uc1.$b13904,1838.0,sentiment_data
...,...,...,...
10206,uiuo.ark+=13960=t3dz0fk4x,1875.0,topic_weights
10207,uc2.ark+=13960=t4zg6w79x,1894.0,topic_weights
10208,uc2.ark+=13960=t1sf2nv5x,1848.0,topic_weights
10209,uiuo.ark+=13960=t3kw5p40t,1835.0,topic_weights


In [23]:
anti['_merge'].value_counts()

left_only     6181
right_only    4030
both             0
Name: _merge, dtype: int64

In [32]:
anti_industry = pd.merge(anti, industry, on = 'HTID')

In [33]:
anti_industry

,HTID,percent_regression,percent_pessimism,percent_optimistic,percent_progress,industry_2_x,industry_3_x,Year,Religion,Science,Political Economy,industry_2_y,industry_3_y
0,nyp.33433081689832,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1810.0,NaN,NaN,NaN,0.000000,0.000000
1,hvd.hn5cyr,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1848.0,NaN,NaN,NaN,0.000000,0.000000
2,uc1.$b283481,0.000000,0.000000,0.000000,0.000000,0.013649,0.013649,1885.0,NaN,NaN,NaN,0.013649,0.013649
3,hvd.32044106334329,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1794.0,NaN,NaN,NaN,0.000000,0.000000
4,uc1.$b13904,0.000253,0.000253,0.000253,0.000253,0.046071,0.050448,1838.0,NaN,NaN,NaN,0.046071,0.050448
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6176,osu.32437121702837,0.000000,0.000000,0.000000,0.000000,0.006479,0.006479,1561.0,NaN,NaN,NaN,0.006479,0.006479
6177,njp.32101067651271,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1800.0,NaN,NaN,NaN,0.000000,0.000000
6178,nnc1.cu55143032,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1755.0,NaN,NaN,NaN,0.000000,0.000000
6179,inu.30000008742250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1730.0,NaN,NaN,NaN,0.000000,0.000000


In [29]:
final_merge

,HTID,percent_regression,percent_pessimism,percent_optimistic,percent_progress,industry_2,industry_3,Religion,Science,Political Economy,Year
0,hvd.32044025716390,0.003229,0.003229,0.003229,0.003229,0.205761,0.235340,0.470864,0.065063,0.464074,1886.0
1,uc2.ark+=13960=t7sn0cd5r,0.001073,0.001073,0.001073,0.001073,0.056452,0.057502,0.413910,0.041589,0.544500,1828.0
2,uiuo.ark+=13960=t83j42z5r,0.000668,0.000668,0.000668,0.000668,0.087561,0.088896,0.289778,0.074732,0.635490,1892.0
3,chi.65460297,0.002482,0.002482,0.002482,0.002482,1.745223,1.938208,0.141372,0.155263,0.703366,1835.0
4,uc1.c034714672,0.000293,0.000293,0.000293,0.000293,0.173269,0.252494,0.236004,0.278340,0.485655,1874.0
...,...,...,...,...,...,...,...,...,...,...,...
162747,mdp.39015063997871,0.000000,0.000000,0.000000,0.000000,0.521087,0.540769,0.142778,0.324604,0.532619,1883.0
162748,nnc1.1002316935,0.001876,0.001876,0.001876,0.001876,0.062852,0.062852,0.353575,0.086502,0.559923,1890.0
162749,mdp.39015033940241,0.000534,0.000534,0.000534,0.000534,0.142026,0.145587,0.380669,0.156957,0.462374,1850.0
162750,hvd.32044023949654,0.000000,0.000000,0.000000,0.000000,0.178859,0.178859,0.357519,0.089237,0.553245,1890.0
